# Moon

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel/iblmodel_moon.ipynb)

In [1]:
%%capture
!pip install iblm --upgrade

In [2]:
import numpy as np
import pandas as pd
import string
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.datasets import make_moons

from iblm import IBLModel


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

### Preparing data sets

In [3]:
seed = 3657
num_train = 300
sample_num = int(num_train/2)

X, y = make_moons(n_samples = 1000, noise=0.05, random_state = seed)
df = pd.DataFrame(data=X, columns=['Feature_1', 'Feature_2']).round(3)
df['target'] = y


df_1 = df[df['target'] == 1].sample(n=sample_num, random_state=seed)
df_0 = df[df['target'] == 0].sample(n=sample_num, random_state=seed)


df_train = pd.DataFrame()
df_len = len(df_1)
for i in range(df_len):
    df1 = pd.DataFrame([df_1.iloc[i]])
    df0 = pd.DataFrame([df_0.iloc[i]])
    df_train = pd.concat([df_train, df1, df0])

df_train['target'] = df_train['target'].astype(int)
df_test = df.drop(df_train.index)

df_train.head()

,Feature_1,Feature_2,target
555,0.259,-0.251,1
595,-0.822,0.584,0
82,0.102,0.025,1
117,-0.386,0.881,0
434,1.941,0.563,1


In [4]:
x_train = df_train.drop('target', axis=1)
y_train = df_train['target']

### Training

In [5]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {
    'columns_name': True,
    'objective': 'classification',
    }
    
iblm = IBLModel(llm_model = llm_model, params=params)

In [11]:
#file_path = '/content/'

#model = iblm.fit(x_train, y_train, model_name = 'pseudodata', file_path=file_path)
model = iblm.fit(x_train, y_train)

In [12]:
# Code of the model created
print(model)

import numpy as np
import pandas as pd

def predict(x):
    df = x.copy()
    
    output = []
    for index, row in df.iterrows():
        if row['Feature_1'] > 0 and row['Feature_2'] < 0:
            y = 0.9
        else:
            y = 0.1
        output.append(y)
    return np.array(output)

# Test the function
data = {
    'Feature_1': [0.259, -0.822, 0.102, -0.386, 1.941],
    'Feature_2': [-0.251, 0.584, 0.025, 0.881, 0.563],
    'target': [1.0, 0.0, 1.0, 0.0, 1.0]
}
df = pd.DataFrame(data)
print(predict(df))


### Prediction

In [13]:
x_test = df_test.drop('target', axis=1)
y_test = df_test['target']

In [14]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

[0.9 0.1 0.1 0.1 0.1]


In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.8357142857142857
Precision: 0.9916317991631799
Recall: 0.6771428571428572
F1 score: 0.8047538200339559
ROC-AUC: 0.8357142857142857
